## Loading Packages

In [5]:
!pip install librosa soundfile numpy sklearn pyaudio
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached sklearn-0.0.post1-py3-none-any.whl
  Using cached PyAudio-0.2.12.tar.gz (42 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: Could not build wheels for pyaudio which use PEP 517 and cannot be installed directly
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Mounting Drive

In [31]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
os.chdir('/content/drive/MyDrive/EmotionDetection')
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/EmotionDetection'

## Loading Libraries

In [30]:
import librosa
import soundfile
import helpers
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/214 [00:00<?, ?B/s]

In [7]:
import warnings
warnings.filterwarnings("ignore")

## Extracting Features

In [9]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        if len(X.shape)>1:
            X=X.flatten()
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

### Mentioning Emotions

In [10]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

## Loading Dataset

In [20]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob(r"/content/drive/MyDrive/EmotionDetection/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        print(file)
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [21]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

/content/drive/MyDrive/EmotionDetection/speech-emotion-recognition-ravdess-data/Actor_06/03-01-01-01-01-02-06.wav
/content/drive/MyDrive/EmotionDetection/speech-emotion-recognition-ravdess-data/Actor_06/03-01-02-02-02-01-06.wav
/content/drive/MyDrive/EmotionDetection/speech-emotion-recognition-ravdess-data/Actor_06/03-01-02-01-02-02-06.wav
/content/drive/MyDrive/EmotionDetection/speech-emotion-recognition-ravdess-data/Actor_06/03-01-01-01-02-02-06.wav
/content/drive/MyDrive/EmotionDetection/speech-emotion-recognition-ravdess-data/Actor_06/03-01-02-01-01-02-06.wav
/content/drive/MyDrive/EmotionDetection/speech-emotion-recognition-ravdess-data/Actor_06/03-01-02-01-02-01-06.wav
/content/drive/MyDrive/EmotionDetection/speech-emotion-recognition-ravdess-data/Actor_06/03-01-02-02-01-01-06.wav
/content/drive/MyDrive/EmotionDetection/speech-emotion-recognition-ravdess-data/Actor_06/03-01-02-02-01-02-06.wav
/content/drive/MyDrive/EmotionDetection/speech-emotion-recognition-ravdess-data/Actor_06

In [22]:
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [23]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [24]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [36]:
from helpers import MLPaccuracy
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [37]:
y_pred=model.predict(x_test)

In [38]:
y_pred

array(['disgust', 'happy', 'happy', 'calm', 'happy', 'calm', 'fearful',
       'fearful', 'fearful', 'calm', 'calm', 'disgust', 'happy', 'calm',
       'fearful', 'calm', 'calm', 'calm', 'fearful', 'disgust', 'happy',
       'disgust', 'disgust', 'calm', 'disgust', 'happy', 'fearful',
       'fearful', 'happy', 'calm', 'happy', 'happy', 'fearful', 'disgust',
       'happy', 'calm', 'happy', 'happy', 'happy', 'calm', 'calm',
       'fearful', 'calm', 'happy', 'happy', 'calm', 'calm', 'calm',
       'fearful', 'happy', 'disgust', 'calm', 'calm', 'happy', 'calm',
       'happy', 'fearful', 'happy', 'calm', 'calm', 'calm', 'calm',
       'disgust', 'calm', 'disgust', 'happy', 'calm', 'disgust',
       'fearful', 'calm', 'disgust', 'fearful', 'fearful', 'disgust',
       'happy', 'calm', 'calm', 'calm', 'calm', 'happy', 'fearful',
       'calm', 'happy', 'fearful', 'happy', 'disgust', 'calm', 'calm',
       'calm', 'calm', 'fearful', 'fearful', 'calm', 'fearful', 'calm',
       'calm', 'cal

## Training SVM Model

In [39]:
#Import svm model
from sklearn import svm
from helpers import SVMAcc


#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(x_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(x_test)

### Printing Accuracies

In [40]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",MLPaccuracy)

Accuracy: 0.7


In [41]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",SVMAcc)

Accuracy: 0.85


### As we can see SVM has Higher Accuracy lets save SVM Model

In [47]:
import joblib
  
  
# Save the model as a pickle in a file
joblib.dump(clf, 'best_model.pkl')
  

['best_model.pkl']

## Testing with own Data

### Using MLP

In [57]:
# file = r"D:\Practice\Emotion Detection using sound\v2code\speech-emotion-recognition-ravdess-data\Actor_01\03-01-01-01-01-01-01.wav"
soundfilepath = r"sound samples/happy.wav"
feature=extract_feature(soundfilepath, mfcc=True, chroma=True, mel=True)

In [52]:
y_pred=model.predict([feature])

In [53]:
y_pred

array(['fearful'], dtype='<U7')

### Using SVM Model

In [54]:
import joblib
import numpy as np
 
# Load the model from the file
modeln = joblib.load('best_model.pkl')

In [55]:
from helpers import modeln
import pandas as pd
import numpy
import sys
sys.path.insert(0,"/content/drive/MyDrive/EmotionDetection")

In [56]:
from helpers import predict
import matplotlib.pyplot as plt

In [65]:
output = predict(modeln, soundfilepath)
output

[{'score': 0.21323682367801666, 'label': 'happiness'},
 {'score': 0.2107013761997223, 'label': 'disgust'},
 {'score': 0.19969108700752258, 'label': 'sadness'},
 {'score': 0.19168557226657867, 'label': 'fear'},
 {'score': 0.18468517065048218, 'label': 'anger'}]

In [64]:
## Final Output
print(output[0]['label'])

'happiness'